In [ ]:
!pip install azure-storage-blob==2.1.0
!pip install pandas --upgrade
#then restart kernel

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep=' | ')

In [ ]:
from azureml.core import Dataset
usql_output = ws.datastores['usql_output']

from azure.datalake.store import core, lib, multithread
adlCreds = lib.auth(tenant_id=usql_output.tenant_id, resource = 'https://datalake.azure.net/')
adls = core.AzureDLFileSystem(adlCreds, store_name=usql_output.store_name)

In [ ]:
from azureml.core import Dataset
from azure.storage.blob import BlockBlobService

logs = ws.datastores['rl_sim_ccb']

bbs = BlockBlobService(
    account_name=logs.account_name,
    account_key=logs.account_key)

In [ ]:
from azure.storage.blob import BlockBlobService
from LogsWalker import Workspace, AppContext, InstanceContext, DayContext, DaySegment
from LogsParser import DsJson
import datetime
import pandas as pd

#Setup some local folder for files storage

ws = Workspace(r'workspace', bbs, adls)

#Get app client
app = ws.get_app(logs.container_name)

#or latest
instance = app.get_instance()


In [ ]:
instance.overview()

In [ ]:
#get given day
oct29 = instance.get_day(datetime.date(2018,10,29))

#get latest day
day = instance.get_day()

In [ ]:
day.overview()

In [ ]:
segment = day.get_segment(0, 8 * 1024 **2)

In [ ]:
#all ccb events stats:
DsJson.ccb_events(segment.read())

In [ ]:
#lookup offset for 11:00 AM and download corresponding segment
segment1100 = day.get_segment(day.lookup(11, 00), 8 * 1024 **2)
DsJson.ccb_events(segment1100.read())

In [ ]:
#ccb stats (dashboard style):
stats = DsJson.ccb_stats(segment.read())

In [ ]:
stats